# 如何在聊天模型中使用少量示例 How to use few shot examples in chat models

本指南介绍如何使用示例输入和输出提示聊天模型。  
为模型提供一些这样的示例称为少量示例，这是一种简单但强大的指导生成方法，在某些情况下可以大大提高模型性能。

关于如何最好地进行少量示例提示似乎没有达成共识，最佳提示编译可能会因模型而异。  
因此，我们提供了少量示例提示模板（如 FewShotChatMessagePromptTemplate）作为灵活的起点，您可以根据需要修改或替换它们。

少量示例提示模板的目标是根据输入动态选择示例，然后在最终提示中格式化示例以提供给模型。

注意：以下代码示例仅适用于聊天模型，因为 FewShotChatMessagePromptTemplates 旨在输出格式化的聊天消息而不是纯字符串。  

## 固定示例
最基本（也是最常见）的少样本提示技术是使用固定提示示例。这样，您可以选择一个链，对其进行评估，并避免担心生产中的其他移动部件。

模板的基本组成部分是：

 - examples：要包含在最终提示中的字典示例列表。
 - example_prompt：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见示例是将每个示例转换为一条人类消息和一条 AI 消息响应，或者一条人类消息后跟一条函数调用消息。
   
以下是一个简单的演示。首先，定义您想要包含的示例：

In [12]:
from langchain_core.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate

examples = [
    {"input":"2+2","output":"4"},
    {"input":"2+3","output":"5"}
]

接下来，将它们组装成小样本提示模板。

In [13]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human","{input}"),
    ("ai","{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='2+2'), AIMessage(content='4'), HumanMessage(content='2+3'), AIMessage(content='5')]


最后，我们组装最终提示，如下所示，将 few_shot_prompt 直接传递到 from_messages 工厂方法中，并将其与模型一起使用：

In [14]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system","你是一位奇妙的数学巫师。"),
    few_shot_prompt,
    ("human","{input}")
])

In [15]:
import os
from dotenv import find_dotenv,load_dotenv

_ = load_dotenv(find_dotenv())

In [16]:
from langchain_openai import ChatOpenAI

chain = (
    final_prompt 
    | ChatOpenAI(
        base_url="http://api.baichuan-ai.com/v1",
        api_key=os.environ["BAICHUAN_API_KEY"],
        model="Baichuan4",
    )
)

print(chain.invoke({"input":"三角形的平方是多少？"}).content)

首先，我们需要澄清一下问题。如果你是指一个具体的三角形的面积，并想要知道这个面积的平方，那么我们需要知道三角形的具体尺寸或者它的面积。如果你是指一个抽象的“三角形”作为一个数学概念的平方，那么这个表述是没有意义的，因为“三角形”不是一个可以进行平方运算的数值。

所以，请提供更多的信息或具体说明你的问题。如果你是指求某个具体三角形的面积，请给出该三角形的底和高，或者其它可以用来计算面积的信息。如果你是在问其他与三角形相关的数学问题，也请详细说明。


## 动态小样本提示

有时您可能希望根据输入从整体集合中仅选择几个示例进行显示。  
为此，您可以用 example_selector 替换传入 FewShotChatMessagePromptTemplate 的示例。  
其他组件与上述相同！我们的动态小样本提示模板如下所示：

 - example_selector：负责为给定输入选择小样本示例（以及返回它们的顺序）。这些组件实现了 BaseExampleSelector 接口。一个常见示例是 vectorstore 支持的 SemanticSimilarityExampleSelector
 - example_prompt：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见示例是将每个示例转换为一条人类消息和一条 AI 消息响应，或者一条人类消息后跟一条函数调用消息。
   
这些再次可以与其他消息和聊天模板组合以组装您的最终提示。

让我们通过 SemanticSimilarityExampleSelector 来看一个示例。  
由于此实现使用 vectorstore 根据语义相似性选择示例，因此我们首先要填充存储。  
由于这里的基本思想是我们要搜索并返回与文本输入最相似的示例，因此我们嵌入提示示例的值而不是考虑键：

In [29]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "母牛对月亮说了什么?", "output": "什么都没说"},
    {
        "input": "写一首关于月亮的诗给我",
        "output": "一首写给月亮，一首写给我自己，我们有什么资格谈论月亮？",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize,embeddings,metadatas=examples)

## 创建 example_selector
创建 vectorstore 后，我们可以创建 example_selector。  
这里我们将单独调用它，并在其上设置 k 以仅获取最接近输入的两个示例。

In [38]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)
# 提示模板将通过传递输入到“select_examples”方法来加载示例
example_selector.select_examples({"input":"月亮"})

[{'input': '写一首关于月亮的诗给我', 'output': '一首写给月亮，一首写给我自己，我们有什么资格谈论月亮？'},
 {'input': '写一首关于月亮的诗给我', 'output': '一首写给月亮，一首写给我自己，我们有什么资格谈论月亮？'}]

## 创建提示模板
我们现在使用上面创建的 example_selector 来组装提示模板。

In [39]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input="What's 3+3?").to_messages())

[HumanMessage(content='2+3'), AIMessage(content='5'), HumanMessage(content='2+3'), AIMessage(content='5')]


并且我们可以将这个小样本聊天消息提示模板传递到另一个聊天提示模板中：

In [40]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位奇妙的数学巫师。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.invoke(input="What's 3+3?"))

messages=[HumanMessage(content='2+3'), AIMessage(content='5'), HumanMessage(content='2+3'), AIMessage(content='5')]


## 与聊天模型一起使用
最后，您可以将模型连接到少样本提示。

In [41]:
chain = final_prompt | ChatOpenAI(
        base_url="http://api.baichuan-ai.com/v1",
        api_key=os.environ["BAICHUAN_API_KEY"],
        model="Baichuan4",
    )

chain.invoke({"input": "What's 3+3?"})

AIMessage(content='6', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 117, 'total_tokens': 119}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f67d7b6a-b3d3-44b7-8a1d-e8d86cf58347-0')